In [1]:
import torch
from emu_base.math.double_krylov import double_krylov, double_krylov_2
from emu_sv.hamiltonian import RydbergHamiltonian
import sys
sys.path.append('./test/utils_testing/')
from utils_dense_hamiltonians import dense_rydberg_hamiltonian

In [2]:
torch.manual_seed(1337)

dtype = torch.complex128
dtype_params = torch.float64
tolerance = 1e-8
N = 6
dt = 0.5

omegas = torch.randn(N, dtype=dtype_params)
deltas = torch.randn(N, dtype=dtype_params)
phis = torch.randn(N, dtype=dtype_params)
interactions = torch.zeros(N, N, dtype=dtype_params)
for i in range(N):
    for j in range(i + 1, N):
        interactions[i, j] = 1 / abs(j - i)

state = torch.randn(2**N, dtype=dtype)
state = state / state.norm()

grad = torch.randn(2**N, dtype=dtype)

ham = RydbergHamiltonian(
    omegas=omegas,
    deltas=deltas,
    phis=phis,
    interaction_matrix=interactions,
    device=state.device,
)

op = lambda x: -1j * dt * (ham * x)

In [3]:
# block diagonal
Hsv = dense_rydberg_hamiltonian(omegas, deltas, phis, interactions)
E = state.unsqueeze(-1) @ grad.conj().unsqueeze(0)
big_mat = torch.block_diag(-1j * dt * Hsv, -1j * dt * Hsv)
sizeH = Hsv.shape[0]
big_mat[:sizeH, sizeH:] = E
big_exp = torch.linalg.matrix_exp(big_mat)
expected_L = big_exp[:sizeH, sizeH:]

print(expected_L)
print(expected_L.shape)

tensor([[ 0.0351+0.0376j, -0.0534-0.0205j,  0.1043-0.0616j,  ...,
          0.0273-0.0592j, -0.0576-0.0014j, -0.0324+0.0662j],
        [ 0.0753-0.0537j, -0.0491+0.0815j, -0.0661-0.1832j,  ...,
         -0.0797-0.0724j, -0.0263+0.0942j,  0.0807+0.0821j],
        [ 0.0276-0.0354j, -0.0012+0.0345j, -0.0520-0.0322j,  ...,
         -0.0161-0.0134j, -0.0006+0.0281j,  0.0076+0.0087j],
        ...,
        [-0.0445+0.0193j,  0.0417-0.0262j, -0.0233+0.0966j,  ...,
          0.0208+0.0755j,  0.0422-0.0463j, -0.0070-0.0907j],
        [-0.0041+0.0435j,  0.0007-0.0531j,  0.0778+0.0846j,  ...,
          0.0926+0.0129j, -0.0261-0.0651j, -0.1097-0.0358j],
        [-0.0055+0.0342j, -0.0051-0.0238j,  0.0385+0.0158j,  ...,
          0.0187+0.0220j,  0.0001-0.0277j, -0.0031-0.0277j]],
       dtype=torch.complex128)
torch.Size([64, 64])


In [4]:
from emu_base.math.krylov_exp import krylov_exp_impl
exp_grad_results = krylov_exp_impl(op, grad, is_hermitian=True, exp_tolerance=tolerance, norm_tolerance=tolerance)
iteration_count = exp_grad_results.iteration_count


lanczos_vectors_even, odd_block, eT, Tb = double_krylov(
    op, grad, state, iteration_count, tolerance
)

even_block = torch.stack(lanczos_vectors_even)
Hess_L = eT[1 : 2 * odd_block.shape[0] : 2, : 2 * even_block.shape[0] : 2]
# L = V_odd @ Hess_L @ V_even*
print("odd:", odd_block.shape)
print("eT:", eT.shape, "Hess_L", Hess_L.shape)
print("even:", even_block.shape)
print("Tb:", Tb.shape)

L = odd_block.mT @ Hess_L @ even_block.conj()

assert torch.allclose(L, expected_L, atol=tolerance)
print(torch.dist(L, expected_L))
print(L)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 1
9 2
10 3
11 4
12 5
13 6
14 7
15 8
16 9
17 10
18 11
19 12
20 13
46
22
odd: torch.Size([14, 64])
eT: torch.Size([46, 46]) Hess_L torch.Size([14, 14])
even: torch.Size([14, 64])
Tb: torch.Size([46, 46])
tensor(5.4432e-10, dtype=torch.float64)
tensor([[ 0.0351+0.0376j, -0.0534-0.0205j,  0.1043-0.0616j,  ...,
          0.0273-0.0592j, -0.0576-0.0014j, -0.0324+0.0662j],
        [ 0.0753-0.0537j, -0.0491+0.0815j, -0.0661-0.1832j,  ...,
         -0.0797-0.0724j, -0.0263+0.0942j,  0.0807+0.0821j],
        [ 0.0276-0.0354j, -0.0012+0.0345j, -0.0520-0.0322j,  ...,
         -0.0161-0.0134j, -0.0006+0.0281j,  0.0076+0.0087j],
        ...,
        [-0.0445+0.0193j,  0.0417-0.0262j, -0.0233+0.0966j,  ...,
          0.0208+0.0755j,  0.0422-0.0463j, -0.0070-0.0907j],
        [-0.0041+0.0435j,  0.0007-0.0531j,  0.0778+0.0846j,  ...,
          0.0926+0.0129j, -0.0261-0.0651j, -0.1097-0.0358j],
        [-0.0055+0.0342j, -0.0051-0.0238j,  0.0385+0.0158j,  ...,
          

In [5]:
lanczos_vectors_even, lanczos_vectors_odd, eT_2, Tb_2 = double_krylov_2(
    op, grad, state, tolerance
)

even_block_2 = torch.stack(lanczos_vectors_even)
odd_block_2 = torch.stack(lanczos_vectors_odd)

Hess_L_2 = eT_2[1 : 2 * odd_block_2.shape[0] : 2, : 2 * even_block_2.shape[0] : 2]

print("odd:", odd_block_2.shape)
print("eT:", eT_2.shape, "Hess_L", Hess_L_2.shape)
print("even:", even_block_2.shape)
print("Tb:", Tb_2.shape)

L_2 = odd_block_2.mT @ Hess_L_2 @ even_block_2.conj()

assert torch.allclose(L_2, expected_L, atol=tolerance)
print(torch.dist(L_2,expected_L))
print(L)

error convergence
15 14 12
error convergence
15 14 12
odd: torch.Size([14, 64])
eT: torch.Size([30, 30]) Hess_L torch.Size([14, 14])
even: torch.Size([14, 64])
Tb: torch.Size([30, 30])
tensor(6.2348e-10, dtype=torch.float64)
tensor([[ 0.0351+0.0376j, -0.0534-0.0205j,  0.1043-0.0616j,  ...,
          0.0273-0.0592j, -0.0576-0.0014j, -0.0324+0.0662j],
        [ 0.0753-0.0537j, -0.0491+0.0815j, -0.0661-0.1832j,  ...,
         -0.0797-0.0724j, -0.0263+0.0942j,  0.0807+0.0821j],
        [ 0.0276-0.0354j, -0.0012+0.0345j, -0.0520-0.0322j,  ...,
         -0.0161-0.0134j, -0.0006+0.0281j,  0.0076+0.0087j],
        ...,
        [-0.0445+0.0193j,  0.0417-0.0262j, -0.0233+0.0966j,  ...,
          0.0208+0.0755j,  0.0422-0.0463j, -0.0070-0.0907j],
        [-0.0041+0.0435j,  0.0007-0.0531j,  0.0778+0.0846j,  ...,
          0.0926+0.0129j, -0.0261-0.0651j, -0.1097-0.0358j],
        [-0.0055+0.0342j, -0.0051-0.0238j,  0.0385+0.0158j,  ...,
          0.0187+0.0220j,  0.0001-0.0277j, -0.0031-0.0277j]],

In [6]:
print(torch.dist(odd_block, odd_block_2))
print(torch.dist(even_block, even_block_2))
#print(torch.dist(Hess_L, Hess_L_2))

tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)


In [7]:
print(Hess_L[1,1])
print(Hess_L_2[1,1])

tensor(-0.6229+0.9755j, dtype=torch.complex128)
tensor(-0.6229+0.9755j, dtype=torch.complex128)


In [8]:
i=14
print(torch.dist(eT[:i,:i], eT_2[:i, :i]))

tensor(2.8209e-10, dtype=torch.float64)


In [9]:
i=21
j=25
print(torch.dist(Tb[i:j, i:j], Tb_2[i:j, i:j]))
print(Tb[i:j, i:j])
print(Tb_2[i:j, i:j])

tensor(0., dtype=torch.float64)
tensor([[ 6.9389e-18-1.2490e+00j,  0.0000e+00+0.0000e+00j,
         -1.4738e+00-1.1449e-16j,  0.0000e+00+0.0000e+00j],
        [ 0.0000e+00+0.0000e+00j, -5.5511e-17-1.0483e+00j,
          0.0000e+00+0.0000e+00j, -1.1828e+00+8.3267e-17j],
        [ 1.4738e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j,
          9.3675e-17-1.2919e+00j,  0.0000e+00+0.0000e+00j],
        [ 0.0000e+00+0.0000e+00j,  1.1828e+00+0.0000e+00j,
          0.0000e+00+0.0000e+00j,  5.5511e-17-1.0604e+00j]],
       dtype=torch.complex128)
tensor([[ 6.9389e-18-1.2490e+00j,  0.0000e+00+0.0000e+00j,
         -1.4738e+00-1.1449e-16j,  0.0000e+00+0.0000e+00j],
        [ 0.0000e+00+0.0000e+00j, -5.5511e-17-1.0483e+00j,
          0.0000e+00+0.0000e+00j, -1.1828e+00+8.3267e-17j],
        [ 1.4738e+00+0.0000e+00j,  0.0000e+00+0.0000e+00j,
          9.3675e-17-1.2919e+00j,  0.0000e+00+0.0000e+00j],
        [ 0.0000e+00+0.0000e+00j,  1.1828e+00+0.0000e+00j,
          0.0000e+00+0.0000e+00j,  5.5511e-1

In [10]:
print(Tb_2.shape)
Tb_2_even = Tb_2[:-2:2,:-2:2]
print(Tb_2_even.shape)

print(Tb.shape)
Tb_even = Tb[:Tb_2.shape[0]-2:2,:Tb_2.shape[1]-2:2]
print(Tb_even.shape)

torch.dist(Tb_2_even, Tb_even)

torch.Size([30, 30])
torch.Size([14, 14])
torch.Size([46, 46])
torch.Size([14, 14])


tensor(1.2573, dtype=torch.float64)

In [11]:
i = 11
print(Tb_even[i:, i:].real)
print(Tb_2_even[i:, i:].real)
print(Tb_even[i:, i:].imag)
print(Tb_2_even[i:, i:].imag)

tensor([[-5.5511e-17, -1.1828e+00,  0.0000e+00],
        [ 1.1828e+00,  5.5511e-17, -1.1125e+00],
        [ 0.0000e+00,  1.1125e+00, -1.3878e-17]], dtype=torch.float64)
tensor([[-5.5511e-17, -1.1828e+00,  0.0000e+00],
        [ 1.1828e+00,  5.5511e-17, -1.1125e+00],
        [ 0.0000e+00,  1.1125e+00,  0.0000e+00]], dtype=torch.float64)
tensor([[-1.0483e+00,  8.3267e-17,  0.0000e+00],
        [ 0.0000e+00, -1.0604e+00,  1.3878e-17],
        [ 0.0000e+00,  0.0000e+00, -1.2573e+00]], dtype=torch.float64)
tensor([[-1.0483e+00,  8.3267e-17,  0.0000e+00],
        [ 0.0000e+00, -1.0604e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]], dtype=torch.float64)


In [12]:
Tb_2_even[12,12]

tensor(5.5511e-17-1.0604j, dtype=torch.complex128)

In [13]:
Tb_2_odd = Tb_2[1::2,1::2]
Tb_odd = Tb[1:Tb_2.shape[0]:2,1:Tb_2.shape[1]:2]
print(torch.dist(Tb_2_odd, Tb_odd))

tensor(0.9083, dtype=torch.float64)


In [14]:
import numpy as np

def arnoldi_iteration(A, b, n: int):
    """Compute a basis of the (n + 1)-Krylov subspace of the matrix A.

    This is the space spanned by the vectors {b, Ab, ..., A^n b}.

    Parameters
    ----------
    A : array_like
        An m × m array.
    b : array_like
        Initial vector (length m).
    n : int
        One less than the dimension of the Krylov subspace, or equivalently the *degree* of the Krylov space. Must be >= 1.
    
    Returns
    -------
    Q : numpy.array
        An m x (n + 1) array, where the columns are an orthonormal basis of the Krylov subspace.
    h : numpy.array
        An (n + 1) x n array. A on basis Q. It is upper Hessenberg.
    """
    eps = 1e-12
    h = torch.zeros(n + 1, n+1, dtype=torch.complex128)
    Q = torch.zeros(b.shape[0], n + 1, dtype=torch.complex128)
    # Normalize the input vector
    Q[:, 0] = b / b.norm()  # Use it as the first Krylov vector
    for k in range(1, n + 1):
        v = A(Q[:, k - 1])  # Generate a new candidate vector
        for j in range(max(0, k-2),k):  # Subtract the projections on previous vectors
            h[j, k - 1] = torch.vdot(Q[:, j], v)
            v -= h[j, k - 1] * Q[:, j]
        nv = v.norm()
        h[k, k - 1] = nv
        if nv > eps:  # Add the produced vector to the list, unless
            Q[:, k] = v / nv
        else:  # If that happens, stop iterating.
            return Q, h
    return Q, h

Q,h = arnoldi_iteration(op,state, 14)
print(Q.shape)
print(h.shape)
print(Q[:,-1].norm())
print(h[12:, 12:])
print(Tb_2_odd.shape)
print(Tb_2_odd[12:, 12:])
print(torch.dist(h[:13, :13], Tb_2_odd[:13, :13]))

torch.Size([64, 15])
torch.Size([15, 15])
tensor(1., dtype=torch.float64)
tensor([[-2.7756e-17-9.1751e-01j, -9.0830e-01-1.1276e-17j,
          0.0000e+00+0.0000e+00j],
        [ 9.0830e-01+0.0000e+00j,  0.0000e+00-6.3227e-01j,
          0.0000e+00+0.0000e+00j],
        [ 0.0000e+00+0.0000e+00j,  1.0613e+00+0.0000e+00j,
          0.0000e+00+0.0000e+00j]], dtype=torch.complex128)
torch.Size([15, 15])
tensor([[ 6.6353e-17-0.9175j, -9.0830e-01+0.0000j,  0.0000e+00+0.0000j],
        [ 9.0830e-01+0.0000j,  0.0000e+00+0.0000j,  0.0000e+00+0.0000j],
        [ 0.0000e+00+0.0000j,  1.0000e+00+0.0000j,  0.0000e+00+0.0000j]],
       dtype=torch.complex128)
tensor(2.8447e-15, dtype=torch.float64)


In [15]:
print(h[3:5,3:5])

tensor([[ 5.5511e-17-1.9601e+00j, -1.4666e+00+1.6653e-16j],
        [ 1.4666e+00+0.0000e+00j,  0.0000e+00-1.3406e+00j]],
       dtype=torch.complex128)
